In [1]:
import pyspark as sp
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,date_format,concat,substring,lit,lpad,to_timestamp,when
import  pandas as pd

In [2]:
spark = SparkSession.builder.getOrCreate()

24/04/18 11:58:24 WARN Utils: Your hostname, Subins-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.65.210.91 instead (on interface en0)
24/04/18 11:58:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/18 11:58:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [163]:
df=spark.read.option("header","true").option("inferSchema", "true").csv("./dataConverted/2014/2014_039620.csv")

ConnectionRefusedError: [Errno 61] Connection refused

In [146]:
df.show(10000)

+-----+-----+----------+-----+------+------+---+-------+-----+-------+-------+------------+--------+-----+------+
| USAF| WBAN|      date| time|   lat|  long|ele|winddir|  sky|visdist|airtemp|dewpointtemp|atmpress|month|season|
+-----+-----+----------+-----+------+------+---+-------+-----+-------+-------+------------+--------+-----+------+
|39620|99999|2014-01-01|00:00|  52.7|-8.917| 20|    150|99999|  20000|    5.1|         3.0|   989.3|    1|Winter|
|39620|99999|2014-01-01|00:00|52.702|-8.925| 14|    150|99999|   9999|    5.0|         3.0|  9999.9|    1|Winter|
|39620|99999|2014-01-01|00:30|52.702|-8.925| 14|    120|99999|   9999|    4.0|         3.0|  9999.9|    1|Winter|
|39620|99999|2014-01-01|01:00|  52.7|-8.917| 20|    140|99999|  20000|    3.8|         2.7|   989.0|    1|Winter|
|39620|99999|2014-01-01|01:00|52.702|-8.925| 14|    140| 1311|   9999|    4.0|         3.0|  9999.9|    1|Winter|
|39620|99999|2014-01-01|01:30|52.702|-8.925| 14|    150| 1311|   9999|    5.0|         3

In [132]:
df = df.withColumn("lat", df["lat"] / 1000)


In [133]:
df = df.withColumn("long", df["long"] / 1000)


In [134]:
df = df.withColumn("airtemp", df["airtemp"] / 10)


In [135]:
df = df.withColumn("dewpointtemp", df["dewpointtemp"] / 10)


In [136]:
df = df.withColumn("atmpress", df["atmpress"] / 10)

In [137]:
df = df.withColumn("date", 
                   concat(substring(col("date"), 1, 4), 
                          lit("-"), 
                          substring(col("date"), 5, 2), 
                          lit("-"), 
                          substring(col("date"), 7, 2)))


In [138]:
df = df.withColumn("time", lpad(col("time"), 4, "0"))

In [139]:
df = df.withColumn("time", 
                   concat(substring(col("time"), 1, 2), 
                          lit(":"),
                          substring(col("time"), 3, 2)))

In [110]:
df = df.withColumn("timestamp", to_timestamp(concat(col("formatted_date"), lit(" "), col("time")), "yyyy-MM-dd HH:mm"))

In [141]:
def get_season(month):
    if 3 <= month <= 5:
        return "Spring"
    elif 6 <= month <= 8:
        return "Summer"
    elif 9 <= month <= 11:
        return "Autumn"
    else:
        return "Winter"


In [143]:
df = df.withColumn("month", col("date").substr(6, 2).cast("int"))
df = df.withColumn("season", when((col("month") >= 3) & (col("month") <= 5), "Spring")
                              .when((col("month") >= 6) & (col("month") <= 8), "Summer")
                              .when((col("month") >= 9) & (col("month") <= 11), "Autumn")
                              .otherwise("Winter"))

In [147]:
summer_df = df.filter(col("season") == "Summer")

In [149]:
correlation_matrix = summer_df.select([col(c) for c in summer_df.columns if c != "season"]).toPandas().corr(numeric_only=True)

In [150]:
print(correlation_matrix)

              USAF  WBAN       lat      long       ele   winddir       sky  \
USAF           NaN   NaN       NaN       NaN       NaN       NaN       NaN   
WBAN           NaN   NaN       NaN       NaN       NaN       NaN       NaN   
lat            NaN   NaN  1.000000 -1.000000 -1.000000  0.046733 -0.091751   
long           NaN   NaN -1.000000  1.000000  1.000000 -0.046733  0.091751   
ele            NaN   NaN -1.000000  1.000000  1.000000 -0.046733  0.091751   
winddir        NaN   NaN  0.046733 -0.046733 -0.046733  1.000000  0.058524   
sky            NaN   NaN -0.091751  0.091751  0.091751  0.058524  1.000000   
visdist        NaN   NaN  0.073502 -0.073502 -0.073502 -0.007767  0.336541   
airtemp        NaN   NaN  0.010412 -0.010412 -0.010412 -0.016142 -0.023340   
dewpointtemp   NaN   NaN -0.014515  0.014515  0.014515 -0.013883 -0.022632   
atmpress       NaN   NaN  0.999999 -0.999999 -0.999999  0.046771 -0.091704   
month          NaN   NaN -0.000384  0.000384  0.000384  0.103615

24/04/14 23:16:53 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1008258 ms exceeds timeout 120000 ms
24/04/14 23:16:53 WARN SparkContext: Killing executors is not supported by current scheduler.
24/04/14 23:34:38 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

In [ ]:
 air_quality_analysis = df.select(
        avg("VisibilityDistance").alias("AvgVisibilityDistance"),
        avg("SkyCeilingHeight").alias("AvgSkyCeilingHeight"),
        avg("AirTemperature").alias("AvgAirTemperature"),
        avg("AtmosphericPressure").alias("AvgAtmosphericPressure")
    )